In [38]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from deeplearning_from_scratch.functions import *
from deeplearning_from_scratch.gradient import *

class SimpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, z)

        return loss

class TwoLayerNet:

    def __init__(self, input_size, hidden_state_size, output_size, weight_init_std=0.01):
        self.params = {}
        
        self.params['W1'] = weight_init_std * \
                                np.random.randn(input_size, hidden_state_size)
        self.params['b1'] = np.zeros(hidden_state_size)
        self.params['W2'] = weight_init_std * \
                                np.random.randn(hidden_state_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        grads = {}
        batch_num = x.shape[0]

        #forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        #backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)

        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads



In [39]:
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_list = []

In [40]:
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

iter_per_epoch = max(1, iters_num / batch_size)

net = TwoLayerNet(input_size=784, hidden_state_size=100, output_size=10)


In [41]:
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # grad = net.numerical_gradient(x_batch, t_batch)
    grad = net.gradient(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2'):
        net.params[key] -= learning_rate * grad[key]

    loss = net.loss(x_batch, t_batch)
    
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = net.accuracy(x_batch, t_batch)
        test_acc = net.accuracy(x_test, t_test)

        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print("iter[{}]'s loss: {}".format(i, loss))
        print("iter[{}]'s train_acc: {}".format(i, train_acc))
        print("iter[{}]'s test_acc: {}".format(i, test_acc))




iter[0]'s loss: 2.275503431092495
iter[0]'s train_acc: 0.09
iter[0]'s test_acc: 0.0887
iter[100]'s loss: 2.259445289848382
iter[100]'s train_acc: 0.13
iter[100]'s test_acc: 0.0983
iter[200]'s loss: 2.0140977237370903
iter[200]'s train_acc: 0.54
iter[200]'s test_acc: 0.4013
iter[300]'s loss: 1.4808023817242417
iter[300]'s train_acc: 0.67
iter[300]'s test_acc: 0.6093
iter[400]'s loss: 1.0552984071597553
iter[400]'s train_acc: 0.78
iter[400]'s test_acc: 0.743
iter[500]'s loss: 0.8719375723701137
iter[500]'s train_acc: 0.86
iter[500]'s test_acc: 0.8024
iter[600]'s loss: 0.7659839252134808
iter[600]'s train_acc: 0.87
iter[600]'s test_acc: 0.8175
iter[700]'s loss: 0.5966647725441913
iter[700]'s train_acc: 0.85
iter[700]'s test_acc: 0.8441
iter[800]'s loss: 0.4904677473190335
iter[800]'s train_acc: 0.9
iter[800]'s test_acc: 0.8558
iter[900]'s loss: 0.5165648168876908
iter[900]'s train_acc: 0.88
iter[900]'s test_acc: 0.8707
iter[1000]'s loss: 0.45392856079384347
iter[1000]'s train_acc: 0.89
it

In [42]:
import matplotlib.pylab as plt

plt.plot(train_loss_list)


In [43]:
plt.plot(train_acc_list)

In [44]:
plt.plot(test_acc_list)